In [142]:
import h5py
import numpy as np
import struct
from tqdm import tqdm

In [48]:
f = h5py.File('/fast_scratch/WatChMaL/debug/dummy_dataset.h5','r')
unchunked = f['event_data']
print(unchunked.chunks)

None


In [34]:
f2 = h5py.File('/fast_scratch/WatChMaL/debug/chunked_dset.h5','r')
chunked = f2['event_data']
print(chunked.chunks)

(1, 40, 40, 38)


In [43]:
%timeit d = chunked[0]

125 µs ± 605 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [44]:
%timeit d = unchunked[0]

122 µs ± 133 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [69]:
f.close()
f2.close()

In [103]:
a = np.ones((15,15),dtype=np.float32)
for i in range(15):
    for j in range(15):
        a[i,j] = i*15 + j
f = h5py.File('dummy.h5','w')
f.create_dataset('data',data=a)
offset = f['data'].id.get_offset()
print(f.keys())
f.close()

<KeysViewHDF5 ['data']>


In [104]:
binary = open('dummy.h5','rb')

In [105]:
binary.seek(offset,0)
struct.unpack('f', binary.read(4))

(0.0,)

In [125]:
struct.unpack('f', binary.read(4))

(20.0,)

In [126]:
binary.close()

In [146]:
f = h5py.File('/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_fulltank_9M_splits_CNN/IWCDmPMT_4pi_fulltank_9M_trainval.h5','r')
offset = f['event_data'].id.get_offset()

In [147]:
binary = open('/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_fulltank_9M_splits_CNN/IWCDmPMT_4pi_fulltank_9M_trainval.h5','rb')

In [148]:
for idx in np.random.choice(np.array(range(len(f['event_data']))), 300):
    data = f['event_data'][idx]
    binary.seek(offset + idx * 40 * 40 * 38 * 4,0)
    for i in range(40):
        for j in range(40):
            for k in range(38):
                assert data[i,j,k] == struct.unpack('f', binary.read(4))
print('success')

success
